In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import albumentations as A
import cv2
import numpy as np
import skimage as ski

import matplotlib.pyplot as plt
import os
import copy

from tqdm import tqdm
from IPython.display import clear_output

import psutil
import pynvml
import sys


sys.path.append('/home/meribejayson/Desktop/Projects/SharkCNN/training_models/dataloaders/')

from test_dataset import SharkDatasetTest as SharkDataset

In [ ]:
lr_model_path = "/home/meribejayson/Desktop/Projects/SharkCNN/training_models/LOGISTIC-REG/train-final-3/lr_weights_train_3.tar"
ann_model_path = "/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-1/ann_weights_train_1.tar"

state_dict_lr = torch.load(lr_model_path)
state_dict_ann = torch.load(ann_model_path)

torch.manual_seed(12)

if not torch.cuda.is_available():
    raise Exception("Couldn't find CUDA")

device = torch.device("cuda")

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)


image_width = 1920
image_height = 1080

In [ ]:
# class ANN(nn.Module):

#     def __init__(self, input_size):
#         super().__init__()
        
#         hidden_layer_num = int(input_size / 2)

#         self.linear1 = nn.Linear(input_size, hidden_layer_num)
#         self.relu1 = nn.ReLU()
#         self.linear2 = nn.Linear(hidden_layer_num, hidden_layer_num)
#         self.relu2 = nn.ReLU()
#         self.linear3 = nn.Linear(hidden_layer_num + input_size, 1)
#         self.sig = nn.Sigmoid()
        

#     def forward(self, x):
#         h = self.linear1(x)
#         h = self.relu1(h)
#         h = self.linear2(h)
#         h = self.relu2(h)
#         h = torch.cat((x, h), dim=1)
#         h = self.linear3(h)
        
#         return self.sig(h)

In [ ]:
class ANN(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        hidden_layer_num = int(input_size / 2)

        self.linear1 = nn.Linear(input_size, hidden_layer_num)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_layer_num + input_size, 1)
        self.sig = nn.Sigmoid()
        

    def forward(self, x):
        h = self.linear1(x)
        h = self.relu1(h)
        h = torch.cat((x, h), dim=1)
        h = self.linear2(h)
        
        return self.sig(h)

In [ ]:
class LogisticRegresion(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        self.linear = nn.Linear(input_size,1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        
        return self.sig(x)

In [ ]:
test_dataset = SharkDataset()

test_data_loader = data.DataLoader(test_dataset, batch_size=500_000, num_workers=1)

In [ ]:
ann_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-4/ann_weights_train_4.tar")
lr_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/LOGISTIC-REG/train-final-6/lr_weights_train_6.tar")

ann_model = ANN(85)
ann_model.load_state_dict(ann_state)
ann_model.to(device)


lr_model = LogisticRegresion(85)
lr_model.load_state_dict(lr_state)
lr_model.to(device)


In [ ]:
def calculate_metrics(preds, data_labels):
    conf_thresholds = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95] 
    precision_varying = []
    recall_varying = []
    f1_score_varying = []

    for thresh in conf_thresholds:
        TP = preds[(preds > thresh) & (data_labels == 1.0)].size(dim=0)
        FP = preds[(preds > thresh) & (data_labels == 0.0)].size(dim=0)
        FN = preds[(preds < thresh) & (data_labels == 1.0)].size(dim=0)

        if(TP + FN + FP == 0):
            return None

        curr_thresh_precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        curr_thresh_recall = TP / (TP + FN) if (TP + FN) != 0 else 0

        curr_thresh_f1_score = (2 * curr_thresh_precision * curr_thresh_recall) / (curr_thresh_recall + curr_thresh_precision) if (curr_thresh_recall + curr_thresh_precision) != 0 else 0


        precision_varying.append(curr_thresh_precision)
        recall_varying.append(curr_thresh_recall)
        f1_score_varying.append(curr_thresh_f1_score)

    MAP_50 = precision_varying[0]
    MAP50_95 = np.mean(np.array(precision_varying))

    best_f1_score_idx = np.argmax(np.array(curr_thresh_f1_score))
    
    f1_score = f1_score_varying[best_f1_score_idx]
    recall = recall_varying[best_f1_score_idx]
    precision = precision_varying[best_f1_score_idx]

    return {
        "MAP50": MAP_50,
        "MAP50_95": MAP50_95,
        "F1_Score": f1_score,
        "Recall": recall,
        "Precision": precision
    } 

In [ ]:
def eval_model(data_loader):
    ann_model.eval()
    lr_model.eval()

    map50_ann = []
    map50_95_ann = []
    f1_score_ann = []
    recall_ann = []
    precision_ann = []

    map50_lr = []
    map50_95_lr = []
    f1_score_lr = []
    recall_lr = []
    precision_lr = []

    curr_iter = 1

    while(True):
        with torch.no_grad():
            for point in data_loader:
                data_inputs = point[:, :-1].to(device).float()
                data_labels = point[:, -1].to(device).float()

                ann_preds = ann_model(data_inputs)
                ann_preds = ann_preds.squeeze(dim=1)

                lr_preds = lr_model(data_inputs)
                lr_preds = lr_preds.squeeze(dim=1)

                metrics_ann_dict = calculate_metrics(ann_preds, data_labels)
                metrics_lr_dict = calculate_metrics(lr_preds, data_labels)

            
                if metrics_ann_dict == None or metrics_lr_dict == None:
                    print(f"Skipped iteration {curr_iter}")
                    continue
            
                map50_ann.append(metrics_ann_dict["MAP50"])
                map50_95_ann.append(metrics_ann_dict["MAP50_95"])
                f1_score_ann.append(metrics_ann_dict["F1_Score"])
                recall_ann.append(metrics_ann_dict["Recall"])
                precision_ann.append(metrics_ann_dict["Precision"])

                map50_lr.append(metrics_lr_dict["MAP50"])
                map50_95_lr.append(metrics_lr_dict["MAP50_95"])
                f1_score_lr.append(metrics_lr_dict["F1_Score"])
                recall_lr.append(metrics_lr_dict["Recall"])
                precision_lr.append(metrics_lr_dict["Precision"])

                MAP_50_ANN = np.mean(np.array(map50_ann))
                MAP_50_95_ANN = np.mean(np.array(map50_95_ann))
                f1_score_ANN = np.mean(np.array(f1_score_ann))
                recall_ANN = np.mean(np.array(recall_ann))
                precision_ANN = np.mean(np.array(precision_ann))

                MAP_50_LR = np.mean(np.array(map50_lr))
                MAP_50_95_LR = np.mean(np.array(map50_95_lr))
                f1_score_LR = np.mean(np.array(f1_score_lr))
                recall_LR = np.mean(np.array(recall_lr))
                precision_LR = np.mean(np.array(precision_lr))

                clear_output(wait=True)
                info = pynvml.nvmlDeviceGetMemoryInfo(handle)
                print(f'Current iteration: {curr_iter}')
                print('='*30)
                print("ANN Metrics")
                print(f"MAP50: {MAP_50_ANN} MAP50-95: {MAP_50_95_ANN} Precision: {precision_ANN} Recall: {recall_ANN} F1 score: {f1_score_ANN}")
                print("\n")
                print("LR Metrics")
                print(f"MAP50: {MAP_50_LR} MAP50-95: {MAP_50_95_LR} Precision: {precision_LR} Recall: {recall_LR} F1 score: {f1_score_LR}")
                print('='*30)
                print(f"CPU Usage: {psutil.cpu_percent()}% GPU memory usage: {int(info.used / info.total)}% \n")
                
        curr_iter += 1

In [ ]:
eval_model(test_data_loader)